In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',-1)


In [53]:
#importing data sets
deliveries  = pd.read_csv('Data_sets/deliveries.csv')
matches  = pd.read_csv('Data_sets/matches.csv')

df = deliveries.copy()
df.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN


In [54]:
#Defining custom functions to fetch balls per boundary, balls per dismissals and get phase

def balls_per_dismissal(balls, dismissals):
    if dismissals==0:
        return balls
    return balls/dismissals
def balls_per_boundary(balls, boundaries):
    if boundaries==0:
        return boundaries
    return balls/boundaries

def get_phase(over_no):
    if over_no<6:
        return "PowerPlay"
    elif over_no<15:
        return "Middle"
    else:
        return "Death"


In [55]:
mdf = matches.copy()
mdf.head(2)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN


### Filters in our problem statement

1. Stadium = 'MA Chidambaram Stadium, Chepauk'
2. Phase = 'Middle Overs' (Assuming 80% of the overs from spinners are in the middle overs)
3. Opposition Team = 'Chennai Super Kings

In [56]:
#Merging matches df and deliveries df to filter the data venue wise
mdf = mdf.rename(columns={'id':'match_id'})
comb_df = pd.merge(df, mdf, on='match_id', how='left')
comb_df.head(2)


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [57]:
#Adding phase details to the merged df
comb_df['over_no'] = comb_df['over'].apply(lambda x:x-1)
comb_df['phase'] = comb_df.over_no.apply(lambda x:get_phase(x))

In [58]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    df=df[df.venue==current_venue]
    df=df[df.phase==current_phase]
    df=df[df.bowling_team==current_opposition]
    df.reset_index(drop=True,inplace =True)
    
    #Feature engineering
    df['isDot'] = df.batsman_runs.apply(lambda x:1 if x==0 else 0)
    df['isOne'] = df.batsman_runs.apply(lambda x:1 if x==1 else 0)
    df['isTwo'] = df.batsman_runs.apply(lambda x:1 if x==2 else 0)
    df['isThree'] = df.batsman_runs.apply(lambda x:1 if x==3 else 0)

    df['isFour'] = df.batsman_runs.apply(lambda x:1 if x==4 else 0)
    df['isSix'] = df.batsman_runs.apply(lambda x:1 if x==6 else 0)


    #Grouping runs, balls, innings, dismissals, sixes and fours for each batter
    runs = pd.DataFrame(df.groupby('batsman')['batsman_runs'].sum()).reset_index().rename(columns={'batsman_runs':'runs'})
    balls = pd.DataFrame(df.groupby('batsman')['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    innings = pd.DataFrame(df.groupby('batsman')['match_id'].nunique()).reset_index().rename(columns={'match_id':'innings'})
    #innings = pd.DataFrame(df.groupby('batsman')['match_id'].apply(lambda x:len(list(x.unique())))).reset_index().rename(columns={'match_id':'innings'})
    dismissals = pd.DataFrame(df.groupby('batsman')['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})

    dots = pd.DataFrame(df.groupby('batsman')['isDot'].sum()).reset_index().rename(columns={'isDot':'dots'})
    sixes = pd.DataFrame(df.groupby('batsman')['isSix'].sum()).reset_index().rename(columns={'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby('batsman')['isFour'].sum()).reset_index().rename(columns={'isFour':'fours'})
    
    #Merging all the above dfs to get a consolidate data frame
    df = pd.merge(innings, runs, on='batsman').merge(balls,on='batsman').merge(dots, on='batsman').merge(dismissals,on='batsman').merge(fours,on='batsman').merge(sixes,on='batsman')
    
    #Calculating RPI, SR, BPD, BPB, dot_percentage
    df['RPI'] = df.apply(lambda x:x['runs']/x['innings'],axis=1)
    df['SR'] = df.apply(lambda x:(x['runs']/x['balls'])*100,axis=1)
    df['BPD'] = df.apply(lambda x:balls_per_dismissal(x['balls'], x['dismissals']),axis=1)
    df['BPB'] = df.apply(lambda x:balls_per_boundary(x['balls'], x['sixes']+x['fours']),axis=1)
    df['dot_percentage'] = df.apply(lambda x:(x['dots']/x['balls'])*100,axis=1)
    return df



In [59]:
df = ByCustom(comb_df, 'MA Chidambaram Stadium, Chepauk','Middle','Chennai Super Kings')
df.head(3)

,batsman,innings,runs,balls,dots,dismissals,fours,sixes,RPI,SR,BPD,BPB,dot_percentage
0,A Mishra,1,14,16,9,1,2,0,14.0,87.5,16.0,8.0,56.250000
1,A Mithun,1,11,8,3,1,2,0,11.0,137.5,8.0,4.0,37.500000
2,A Symonds,1,27,27,12,0,2,1,27.0,100.0,27.0,9.0,44.444444


In [60]:
wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45 #Values after applying Pair-wise matrix method and Saaty scale

## Filtering to remove outliers

In [61]:
df = df[(df.innings>=2) & (df.balls>=20)]
df.head(3)

,batsman,innings,runs,balls,dots,dismissals,fours,sixes,RPI,SR,BPD,BPB,dot_percentage
4,AB de Villiers,4,88,61,17,1,10,1,22.0,144.262295,61.0,5.545455,27.868852
6,AD Mathews,2,42,36,12,0,2,2,21.0,116.666667,36.0,9.000000,33.333333
7,AD Russell,2,37,27,7,1,3,1,18.5,137.037037,27.0,6.750000,25.925926


## Calculation - Normalizing all values to same dimensions

In [62]:
# Topsis Method
#Step 1 - Calculating squares of SR RPI, BPD, Dot percentage
df['calc_SR'] = df['SR'].apply(lambda x:x*x)
df['calc_RPI'] = df['RPI'].apply(lambda x:x*x)
df['calc_BPD'] = df['BPD'].apply(lambda x:x*x)
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x:x*x)

#Step 2 : Calculating sum of squares of each param and taking square root of each param

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df[['calc_SR','calc_RPI', 'calc_BPD','calc_dot_percentage']].sum(axis=0))

#Step 3: Divide each value for the respective param
df['calc_SR'] = df['calc_SR'].apply(lambda x:x/sq_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x:x/sq_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x:x/sq_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x:x/sq_dot_percentage)

#Step 4: Multiply the weights to the above obtained values
df['calc_SR'] = df['calc_SR'].apply(lambda x:x*wt_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x:x*wt_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x:x*wt_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x:x*wt_dot_percentage)

#Step 5 : Calculate the ideal best and worst solution (max and min)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df['calc_dot_percentage']), max(df['calc_dot_percentage'])

## Calculation - Comparision against the best & worst values

In [63]:
#Calculating the separation measure for each row

#Step 6 : Calculating the squares of difference between value and best value
df['dev_best_SR'] = df['calc_SR'].apply(lambda x:(x-best_sr)**2)
df['dev_best_RPI'] = df['calc_RPI'].apply(lambda x:(x-best_rpi)**2)
df['dev_best_BPD'] = df['calc_BPD'].apply(lambda x:(x-best_bpd)**2)
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x:(x-best_dot_percentage)**2)

#Step 7 : Calculating the Postive ideal separation

df['dev_best_sqrt']=np.sqrt(df[['dev_best_SR','dev_best_RPI','dev_best_BPD','dev_best_dot_percentage']].sum(axis=1))

#Step 8 : Calculating the squares of difference between value and worst value
df['dev_worst_SR'] = df['calc_SR'].apply(lambda x:(x-worst_sr)**2)
df['dev_worst_RPI'] = df['calc_RPI'].apply(lambda x:(x-worst_rpi)**2)
df['dev_worst_BPD'] = df['calc_BPD'].apply(lambda x:(x-worst_bpd)**2)
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x:(x-worst_dot_percentage)**2)

#Step 9 : Calculating the Negative ideal separation

df['dev_worst_sqrt']=np.sqrt(df[['dev_worst_SR','dev_worst_RPI','dev_worst_BPD','dev_worst_dot_percentage']].sum(axis=1))


## Calculating relative closeness score

In [64]:
df['score'] =df.apply(lambda x:x['dev_worst_sqrt']/(+x['dev_best_sqrt']+x['dev_worst_sqrt']),axis=1)

## End Result: Top players for our required role

In [65]:
df[[ 'batsman','innings', 'runs', 'balls', 'dismissals', 'dot_percentage', 'score']].sort_values(by='score', ascending = False).reset_index(drop = True).head(25)

,batsman,innings,runs,balls,dismissals,dot_percentage,score
0,SE Marsh,3,104,54,1,18.518519,0.868050
1,SR Watson,4,121,70,2,22.857143,0.753628
2,AB de Villiers,4,88,61,1,27.868852,0.713668
3,G Gambhir,3,86,73,1,28.767123,0.676621
4,NV Ojha,3,64,41,2,29.268293,0.656592
5,Y Venugopal Rao,3,67,51,1,31.372549,0.646797
6,MS Bisla,3,89,66,2,30.303030,0.641446
7,AD Russell,2,37,27,1,25.925926,0.631002
8,DA Miller,3,38,32,1,25.000000,0.598423
9,V Kohli,7,175,150,4,33.333333,0.588780
